In [1]:
import pandas as pd
import pyodbc

# Initialize connection parameters

In [2]:
DB = {'servername': 'MSI',
      'source': 'AdventureWorks2019',
      'destination':'DW',
      'user':'user1',
      'pass':'123456'}

# Create connection

In [41]:
#SourceDB
sourceConn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['source'] +';UID='+ DB['user']+';PWD='+DB['pass']+ ';Trusted_Connection=yes')

#DestinationDB
destConn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + DB['servername'] + ';DATABASE='+DB['destination'] +';UID='+ DB['user']+';PWD='+DB['pass']+ ';Trusted_Connection=yes')
cursor=destConn.cursor()

In [ ]:
df=pd.read_sql('SELECT * FROM Purchasing.ShipMethod',sourceConn,index_col="ShipMethodID")

In [ ]:
# df = pd.read_sql('SELECT * FROM Purchasing.[PurchaseOrderHeader]',sourceConn)
# df['ModifiedDate'] =pd.to_datetime(df['ModifiedDate']).dt.date
# date=df['ModifiedDate'].drop_duplicates().reset_index(drop=True)
# print(date)

# Extract data from source DB to pandas DataFrames

In [35]:
Tables= [('Purchasing.ShipMethod','ShipMethodID'),('Production.Product','ProductID'),('Purchasing.Vendor','BusinessEntityID'),('Purchasing.PurchaseOrderHeader','PurchaseOrderID'),
         ('Purchasing.PurchaseOrderDetail',['PurchaseOrderID','PurchaseOrderDetailID']), 
         ('Person.Person','BusinessEntityID'),('Sales.Customer','CustomerID'),('Sales.Store','BusinessEntityID'),('Sales.SalesTerritory','TerritoryID'),
         ('Person.CountryRegion','CountryRegionCode'),('Sales.SalesOrderHeader','SalesOrderID'),('Sales.SalesOrderDetail',['SalesOrderID','SalesOrderDetailID'])
        ]

In [ ]:
# df=pd.read_sql('SELECT BusinessEntityID FROM HumanResources.Employee',sourceConn,index_col='BusinessEntityID')
# df


In [ ]:
dfs={}
for item in Tables:
    dfs[item[0]]=pd.read_sql(f'SELECT * FROM {item[0]}',sourceConn,index_col=item[1])



## Transform staged data

In [54]:
#columns to extract in each table
Columns= [('Purchasing.ShipMethod','Name'),('Production.Product',['Name','Color','Size','Class','Style']),('Purchasing.Vendor',['Name','CreditRating']),
         ('Purchasing.PurchaseOrderHeader',['Status','EmployeeID','VendorID','ShipMethodID','OrderDate','ShipDate']),
         ('Purchasing.PurchaseOrderDetail',['DueDate','OrderQty','ProductID','LineTotal','ReceiveQty','RejectQty','StockedQty']), 
         ('Person.Person',['BusinessEntityID']),('Sales.Customer','CustomerID'),('Sales.Store','BusinessEntityID'),('Sales.SalesTerritory','TerritoryID'),
         ('Person.CountryRegion','CountryRegionCode'),('Sales.SalesOrderHeader','SalesOrderID'),('Sales.SalesOrderDetail',['SalesOrderID','SalesOrderDetailID'])
        ]

In [66]:
df=pd.DataFrame(dfs['Production.Product'][['Name','Color','Size','Class','Style']])
df

,Name,Color,Size,Class,Style
ProductID,,,,,
1,Adjustable Race,None,None,None,None
2,Bearing Ball,None,None,None,None
3,BB Ball Bearing,None,None,None,None
4,Headset Ball Bearings,None,None,None,None
316,Blade,None,None,None,None
...,...,...,...,...,...
995,ML Bottom Bracket,None,None,M,None
996,HL Bottom Bracket,None,None,H,None
997,"Road-750 Black, 44",Black,44,L,U


In [ ]:
for index, row in pd.DataFrame(df['Name']).iterrows():
      cursor.execute('''INSERT INTO dbo.DimShipMethod (ShipMethodID,Name) values(?,?)''',index, row.Name)
      
cursor.commit()
